In [6]:
import os
import numpy as np
import pandas as pd
import librosa
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [7]:
data_dir="C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small"
file_path="C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small/000/000002.mp3"
output_path = 'C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/460jFinalProject/extracted_features_combined.csv'


In [8]:
# Paths to the datasets
# tracks_path = '/Users/dkamboj6/Recitation/460j_fma_dataset/fma_metadata/tracks.csv'
# genres_path = '/Users/dkamboj6/Recitation/460j_fma_dataset/fma_metadata/genres.csv'
tracks_path=f"C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_metadata/tracks.csv"
genres_path= f"C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_metadata/genres.csv"
# Load data
tracks = pd.read_csv(tracks_path, index_col=0, header=[0, 1])
genres = pd.read_csv(genres_path)


In [9]:
def load_genre_metadata(tracks_path):
    # Load track metadata for genre labels
    tracks_df = pd.read_csv(tracks_path, index_col=0, header=[0, 1])
    genre_labels = tracks_df['track']['genre_top']  # Adjust if the column names are different
    return genre_labels.to_dict()

# Define the path to your tracks.csv metadata file
genre_dict = load_genre_metadata(tracks_path)


In [10]:

def extract_all_features(file_path):
    try:
        # Load the audio file
        audio, sample_rate = librosa.load(file_path, sr=None)

        # Compute the Discrete Fourier Transform (DFT) features
        dft = np.abs(librosa.stft(audio))
        dft_features = np.mean(dft, axis=1)

        # Extract other features using librosa
        mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
        mfccs_processed = np.mean(mfcc.T, axis=0)
        stft = np.abs(librosa.stft(audio))
        contrast = librosa.feature.spectral_contrast(S=stft, sr=sample_rate)
        contrast_processed = np.mean(contrast.T, axis=0)
        chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
        chroma_processed = np.mean(chroma.T, axis=0)
        mel = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
        mel_processed = np.mean(mel.T, axis=0)

        # Combine all features into one array
        combined_features = np.hstack([mfccs_processed, contrast_processed, chroma_processed, mel_processed, dft_features])

    except Exception as e:
        print(f"Error encountered while parsing file: {file_path}, error: {e}")
        return None, None

    return combined_features, file_path.split('/')[-1].split('.')[0]  # Return features and track ID


In [43]:

def process_audio_files(base_directory, genre_dict):
    features_list = []
    track_ids = []
    genres = []

    # Walk through all files in the directory structure
    for subdir, dirs, files in os.walk(base_directory):
        for file in files:
            if file.endswith('.mp3'):
                file_path = os.path.join(subdir, file)
                features, track_id = extract_all_features(file_path)
                base_name = os.path.basename(file_path)  # Gets '000002.mp3'
                track_id_str = os.path.splitext(base_name)[0]  # Splits off the '.mp3', resulting in '000002'
                track_id = int(track_id_str)
                # print(track_id)

                if features is not None and track_id in genre_dict:
                    features_list.append(features)
                    track_ids.append(track_id)
                    genres.append(genre_dict[track_id])  # Fetch the genre from the dictionary

    return features_list, track_ids, genres




In [44]:
# Process all audio files and collect features along with genres
all_features, all_track_ids, all_genres = process_audio_files(data_dir, genre_dict)

# Create a DataFrame from the extracted features, track IDs, and genres
features_df = pd.DataFrame(all_features, index=all_track_ids)
features_df['genre'] = all_genres  # Add genre as a column
features_df.index.name = 'track_id'


C:\Users\divya\AppData\Local\Temp\ipykernel_26972\3622001746.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(file_path, sr=None)
c:\Users\divya\OneDrive\Documents\UTAUSTIN\Semester 6\460j\460jFinalProject\.venv\Lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error encountered while parsing file: C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small\098\098565.mp3, error: 
Error encountered while parsing file: C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small\098\098567.mp3, error: 
Error encountered while parsing file: C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small\098\098569.mp3, error: 
Error encountered while parsing file: C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small\099\099134.mp3, error: 


c:\Users\divya\OneDrive\Documents\UTAUSTIN\Semester 6\460j\460jFinalProject\.venv\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Error encountered while parsing file: C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small\108\108925.mp3, error: 
Error encountered while parsing file: C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small\133\133297.mp3, error: 


In [47]:
display(features_df)

,0,1,2,3,4,5,6,7,8,9,...,1176,1177,1178,1179,1180,1181,1182,1183,1184,genre
track_id,,,,,,,,,,,,,,,,,,,,,
2,-122.713936,117.760094,-42.334183,38.061005,-23.012323,23.920383,-11.944865,13.687291,-7.294582,4.041143,...,0.000032,0.000033,0.000039,0.000048,0.000078,0.000110,0.000089,0.000050,0.000029,Hip-Hop
5,-162.232422,131.435989,-18.426781,51.631371,-15.444844,23.860094,-7.867664,10.686234,-0.314591,12.864646,...,0.000039,0.000041,0.000060,0.000092,0.000102,0.000146,0.000153,0.000072,0.000038,Hip-Hop
10,-75.631538,154.537613,-63.849487,22.506426,2.425359,10.459981,-13.007332,9.818939,-7.640783,3.136315,...,0.000056,0.000066,0.000069,0.000083,0.000095,0.000103,0.000103,0.000072,0.000058,Pop
140,-322.997864,133.684906,15.497437,54.755615,8.718211,37.022747,-2.116411,15.265619,6.175477,-2.599205,...,0.000041,0.000035,0.000031,0.000038,0.000050,0.000052,0.000034,0.000024,0.000022,Folk
141,-253.314560,157.769379,-6.081675,26.920168,6.813843,15.842272,-12.930041,10.373278,-0.609945,-8.300909,...,0.000029,0.000025,0.000026,0.000048,0.000095,0.000081,0.000036,0.000029,0.000023,Folk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154308,-285.825165,155.782806,-7.308323,2.493736,-4.307317,1.880550,-0.723518,2.616793,7.539652,1.717390,...,0.000013,0.000016,0.000018,0.000021,0.000046,0.000094,0.000080,0.000025,0.000010,Hip-Hop
154309,-325.746490,94.565117,12.300579,-1.013848,-16.124542,4.808063,-9.805314,-7.798704,-13.323016,-5.347288,...,0.000015,0.000016,0.000021,0.000031,0.000033,0.000028,0.000019,0.000012,0.000010,Hip-Hop
154413,-264.023285,173.237167,15.978307,47.577950,-18.508392,20.177227,18.945585,15.323055,15.927598,0.358214,...,0.001181,0.001096,0.000982,0.000926,0.000939,0.001264,0.001563,0.001546,0.001409,Pop


In [48]:
# Save the DataFrame to a CSV file
features_df.to_csv(output_path)

print(f"All features with genre saved to {output_path}")


All features with genre saved to C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/460jFinalProject/extracted_features_combined.csv


Train on Neural Network

In [152]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from torch.utils.data import Dataset, TensorDataset


In [153]:
# Setup device: use CUDA if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")

Training on cuda


In [154]:
data_df =pd.read_csv(output_path)

In [155]:
encoder = LabelEncoder()
data_df['genre_encoded'] = encoder.fit_transform(data_df['genre'])

In [156]:
X = data_df.drop(['genre', 'genre_encoded'], axis=1).values
y = data_df['genre_encoded'].values

In [157]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)



In [158]:
train_dataset

In [159]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AdvancedGenreClassifier(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes, dropout_rate=0.5):
        super(AdvancedGenreClassifier, self).__init__()
        # Define the first layer
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.bn1 = nn.BatchNorm1d(hidden_sizes[0])
        
        # Define the second layer
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.bn2 = nn.BatchNorm1d(hidden_sizes[1])

        # Define the third layer
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.bn3 = nn.BatchNorm1d(hidden_sizes[2])
        
        # Define the output layer
        self.fc4 = nn.Linear(hidden_sizes[2], num_classes)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

# Parameters
input_size = X_train.shape[1]
hidden_sizes = [512, 256, 128]  # More complex and deeper network
num_classes = len(np.unique(y))
dropout_rate = 0.5  # Dropout rate of 50%

# Initialize the advanced model
advanced_model = AdvancedGenreClassifier(input_size, hidden_sizes, num_classes, dropout_rate).to(device)


In [160]:
class CNN1D(nn.Module):
    def __init__(self, num_features, num_classes):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * num_features, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

num_features = X_train.shape[1]
num_classes = len(np.unique(y_train))
cnn_model = CNN1D(num_features, num_classes).to(device)


In [166]:
import torch.optim as optim
from sklearn.metrics import roc_auc_score

# Setup the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(advanced_model.parameters(), lr=0.001)

# Training and evaluation loop, similar to the simpler model
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)


In [164]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score
def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Binarize the labels for multi-class setting
    all_labels_binary = label_binarize(all_labels, classes=np.arange(num_classes))
    
    # Calculate ROC AUC score for One-vs-Rest
    if all_preds[0].shape[0] == num_classes:
        all_preds_array = np.array(all_preds)
    else:
        all_preds_array = np.column_stack(all_preds)
    
    auc_score = roc_auc_score(all_labels_binary, all_preds_array, multi_class="ovr")
    accuracy = correct / total
    return auc_score, accuracy


In [167]:
def train_model(model, train_loader, test_loader, device, epochs=100):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    acc=0

    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        total = correct = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        acc= max(acc,100 * correct / total)

        # print(f'Epoch {epoch+1}: Accuracy: {100 * correct / total}%')
    return(acc)

# Train CNN1D model
acc = train_model(cnn_model, train_loader, test_loader, device)
print(f"finished cnn_model with best accuracy {acc}")
# Train LSTM model
# Make sure input to LSTM is reshaped or processed as sequence data
# Train the LSTM model
acc=train_model(advanced_model, train_loader, train_loader, device)
print(f"finished advanced_model with best accuracy {acc}")


finished cnn_model with best accuracy 54.59662288930581
finished advanced_model with best accuracy 77.21657544956997


XG BOOST : TO do

In [139]:
data_df=pd.read_csv(output_path)

In [140]:

# Assume features_df contains features and a 'genre' column
X = data_df.drop(['genre'], axis=1).values
y = data_df['genre'].values

# Encode the genre labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [141]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

param_grid = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'subsample': [0.6, 0.8, 1.0]
}


In [144]:
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=25,  # Number of parameter settings that are sampled
    scoring='roc_auc_ovr',  # Multi-class AUC score
    cv=5,  # 3-fold cross-validation
    verbose=2,
    random_state=42,
)

# Perform the random search
random_search.fit(X_train_scaled, y_train)


Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=7, n_estimators=500, subsample=0.6; total time= 4.6min
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=7, n_estimators=500, subsample=0.6; total time= 4.6min
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=7, n_estimators=500, subsample=0.6; total time= 4.6min
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=7, n_estimators=500, subsample=0.6; total time= 4.7min
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=7, n_estimators=500, subsample=0.6; total time= 4.7min
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimators=500, subsample=0.6; total time= 1.5min
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimators=500, subsample=0.6; total time= 1.5min
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimators=500, subsample=0.6; total time= 1.5min
[CV] END c

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='mlogloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learnin...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   n_iter=25,
                   param_distributions={'colsample_bytree': [0.5, 0.7, 1.0],
                                        'learning_rate': [0.01, 0.05, 0.1],
                                        'max_depth': [3, 5, 7],
                                        'n_estimators': [100, 300, 500],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=42, scoring='roc_auc_ovr', verbose=2)

In [145]:
best_model = random_search.best_estimator_
best_params = random_search.best_params_
test_auc = best_model.score(X_test_scaled, y_test)

print("Best parameters found: ", best_params)
print("Test set AUC: ", test_auc)

Best parameters found:  {'subsample': 1.0, 'n_estimators': 500, 'max_depth': 5, 'learning_rate': 0.1, 'colsample_bytree': 0.7}
Test set AUC:  0.6128830519074422


In [147]:
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', **best_params)
model.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [148]:
y_pred_prob = model.predict_proba(X_test_scaled)

# Compute AUC for each class and average
auc_scores = []
for i in range(len(encoder.classes_)):
    class_auc = roc_auc_score(y_test == i, y_pred_prob[:, i])
    auc_scores.append(class_auc)
average_auc = np.mean(auc_scores)

print("AUC Scores per class: ", auc_scores)
print("Average AUC: ", average_auc)

AUC Scores per class:  [0.8853981580194266, 0.8651766621768676, 0.9354174190552702, 0.947853249609537, 0.9456587300390549, 0.9248455442341906, 0.8229516794202484, 0.9283157206292382]
Average AUC:  0.9069521453979792


Pre trained model

In [ ]:
data_df=pd.read_csv(output_path)